In [ ]:
from stochastic_volatility_model import *

# 1. Exemples de trajectoires

In [ ]:
N = 10000
T = 100
alpha_cst= 0
beta_cst = 0.9702
mu_cst = 0
W_cst = 1

> **Méthode A**

In [ ]:
show_random_trajectory_stochastic_volatility_model(T)

> **Méthode B**

In [ ]:
# on génère la trajectoire 
hidden_states, observations = generate_trajectory_stochastic_volatility_model(T)

# on les plots
show_a_trajectory_stochastic_volatility_model(T,hidden_states, observations)

# 2. Quelques runs du SIR 

> **Méthode A**

In [ ]:
show_SIR_random_trajectory_stochastic_volatility_model(T,N,alpha_cst,W_cst,beta_cst,mu_cst)

> **Méthode B**

In [ ]:
# On génère les hidden states et observations
hidden_states, observations = generate_trajectory_stochastic_volatility_model(T)

# On applique le SIR et on le plot  
show_SIR_trajectory_stochastic_volatility_model(T,N,hidden_states,observations,alpha_cst,W_cst,beta_cst,mu_cst)

# 3. Quelques runs du Storvik

> **Méthode A**

In [ ]:
show_random_storvik_SIR_stochastic_volatility_model(T,N)

> **Méthode B**

In [ ]:
# On génère la trajectoire et les observations
hiden_states,observations = generate_trajectory_stochastic_volatility_model(T)

# On applique et affiche le filtre de Storvik
show_storvik_SIR_stochastic_volatility_model(T,N,hiden_states,observations)

# 4. Estimation des paramètres du modèle

> **Non moyenné**

In [ ]:
T = 1000
N = 1000
show_random_stovik_SIR_and_parameters_stochastic_volatility_model(T,N)

> **Moyenné**

In [ ]:
from stochastic_volatility_model import *
T = 1000
nbr_iteration = 10
N = 100
n_W,n_alpha,n_beta,n_mu,n_W_parametre = run_n_storvik_SIR_stochastic_volatility_model(T,N,nbr_iteration)

show_n_runs_stochastic_volatility_model(n_W,n_W_parametre,n_alpha,n_beta,n_mu)

# 5. Testes avec les paramètres estimés

In [ ]:
estimated_W, estimated_alpha, estimated_beta, mu_estimated = parameters_estimations(n_W,n_W_parametre, n_alpha, n_beta, n_mu)
print(estimated_W, estimated_alpha, estimated_beta, mu_estimated)
show_SIR_random_trajectory_stochastic_volatility_model(T,N,estimated_alpha,estimated_W,estimated_beta,mu_estimated)

# PLS

In [ ]:
from stochastic_volatility_model import * 

N = 1000
T = 1000

# On crée la trajectoire 
hidden_states,Y = generate_trajectory_stochastic_volatility_model(T)

# On run le Storvik dessus 
X, W, W_estimated, alpha_estimated, beta_estimated, mu_estimated = storvik_SIR_stochastic_volatility_model(T,Y,N)
hidden_states_estimated = np.sum(W*X,axis=1)

# On smooth l'estimation de la trajectoire
W_smoothed, X_smoothed = backward_filter_stochastic_volatility_model(T,N,W,X,W_estimated, alpha_estimated, beta_estimated, mu_estimated)
hidden_states_estimated_smoothed = np.sum(W_smoothed*X_smoothed,axis=1) 

L_t = np.arange(T)
plt.figure(figsize=(20,8))
sns.scatterplot(x=L_t,y=hidden_states_estimated,label='Storvik\' filter',color='grey')
sns.lineplot(x=L_t,y=hidden_states_estimated_smoothed,label='backward smoother')
sns.lineplot(x=L_t,y=hidden_states,label='hidden state',color='black')
plt.xlabel('Temps')
plt.title("Backward smoother with "+str(N)+' particles on stochastic volatility model', fontsize=30,fontname='Times New Roman')  
plt.legend()
plt.show() 

In [ ]:
print('W = ',np.sum(W_estimated[-1]*W[-1]))
print('alpha = ',np.sum(alpha_estimated[-1]*W[-1]))
print('beta = ',np.sum(beta_estimated[-1]*W[-1]))

In [ ]:
W_cst = np.sum(W_estimated[-1]*W[-1])
alpha_cst = np.sum(alpha_estimated[-1]*W[-1])
beta_cst = np.sum(beta_estimated[-1]*W[-1])

SIR_W,SIR_X = SIR_stochastic_volatility_model(T,Y,N,W_cst,alpha_cst,beta_cst,0)
SIR_hidden_states_estimated = np.sum(SIR_W*SIR_X,axis=1)

L_t = np.arange(T)
plt.figure(figsize=(20,8))
#sns.scatterplot(x=L_t,y=hidden_states_estimated,label='Storvik\' filter',color='grey')
sns.lineplot(x=L_t,y=SIR_hidden_states_estimated,label='SIR filter with estimated parameters',color='blue')
#sns.lineplot(x=L_t,y=hidden_states_estimated_smoothed,label='PLS')
sns.lineplot(x=L_t,y=hidden_states,label='hidden state',color='black')
plt.xlabel('Temps')
plt.title("Storvik\'s filter, PLS and SIR with "+str(N)+' particles on stochastic volatility model', fontsize=30,fontname='Times New Roman')  
plt.legend()
plt.show()

# FFBS 

In [ ]:
from stochastic_volatility_model import *

N = 100
T = 100
alpha_par = 0
beta_par = 0.9702
mu_par = 0
W_par = 1

# On génère la trajectoire 
hidden_states, observations = generate_trajectory_stochastic_volatility_model(T)

# On estime avec un SIR classique 
weights, X_estimated = SIR_stochastic_volatility_model(T,observations,N,W_par,alpha_par,beta_par,mu_par)
hidden_states_estimated = np.sum(weights*X_estimated,axis=1)

# On smooth la trajectoire obtenue
weights_smoothed = FBS_stochastic_volatility_model(T=T,N=N,weights=weights,X=X_estimated,W_par=W_par,alpha_par=alpha_par,beta_par=beta_par)
hidden_states_estimated_smoothed = np.sum(weights_smoothed*X_estimated,axis=1)

# On plot 
L_t = np.arange(T)
plt.figure(figsize=(20,8))
sns.lineplot(x=L_t,y=hidden_states_estimated,label='SIR',marker='o',color='blue')
sns.lineplot(x=L_t,y=hidden_states_estimated_smoothed,marker='o',label='FFBS')
sns.lineplot(x=L_t,y=hidden_states,label='hidden state',marker='o',color='black')
plt.xlabel('Temps')
plt.title("SIR and FFBS with "+str(N)+' particles on stochastic volatility model', fontsize=30,fontname='Times New Roman')  
plt.legend()
plt.show()

# Storvik puis SIR avec les paramètres estimés 

In [ ]:
from stochastic_volatility_model import * 

N = 1000
T = 1000

print('On crée la trajectoire') 
hidden_states,Y = generate_trajectory_stochastic_volatility_model(T)

print('On run le Storvik dessus') 
X, W, W_estimated, alpha_estimated, beta_estimated, mu_estimated = storvik_SIR_stochastic_volatility_model(T,Y,N)
hidden_states_estimated = np.sum(W*X,axis=1)

print('Les paramètres estimés') 
W_cst = np.sum(W_estimated[-1]*W[-1],axis=1)
alpha_cst = np.sum(alpha_estimated[-1]*W[-1],axis=1)
beta_cst = np.sum(beta_estimated[-1]*W[-1],axis=1)
print('W = ',W_cst)
print('alpha = ',alpha_cst)
print('beta = ',beta_cst)

print('On run le SIR') 
SIR_W,SIR_X = SIR_stochastic_volatility_model(T,Y,N,W_cst,alpha_cst,beta_cst,0)
SIR_hidden_states_estimated = np.sum(SIR_W*SIR_X,axis=1)

print('On plot')
L_t = np.arange(T)
plt.figure(figsize=(20,8))
#sns.scatterplot(x=L_t,y=hidden_states_estimated,label='Storvik\' filter',color='grey')
sns.lineplot(x=L_t,y=SIR_hidden_states_estimated,label='SIR filter with estimated parameters',color='blue')
sns.lineplot(x=L_t,y=hidden_states,label='hidden state',color='black')
plt.xlabel('Temps')
plt.title("Storvik\'s filter, SIR with "+str(N)+' particles on stochastic volatility model', fontsize=30,fontname='Times New Roman')  
plt.legend()
plt.show()